In [70]:
from bs4 import BeautifulSoup
import os
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
from flask import Flask

In [71]:
def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser('chrome', **executable_path, headless=False)


def scrape():
    
    browser = init_browser()
    mars_collection = {}
    url1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url1)
    html = browser.html
    response = requests.get(url1)
    soup1 = BeautifulSoup(response.text, 'html.parser')
    
    mars_collection["news_title"] = soup1.find('div', class_='content_title').text.strip()
    mars_collection["news_p"] = soup1.find('div', class_='rollover_description_inner').text.strip()
    
    url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url2)
    browser.click_link_by_id('full_image')
    time.sleep(2)
    html = browser.html
    img_soup = BeautifulSoup(html, "html.parser")
    img_url = img_soup.find('img', class_='fancybox-image')['src']
    mars_collection["featured_img_url"] = "https://www.jpl.nasa.gov" + img_url
    
    url3 = 'https://twitter.com/marswxreport?lang=en'
    response = requests.get(url3)
    soup2 = BeautifulSoup(response.text, 'html.parser')
    mars_collection["mars_weather"] = soup2.find('div', class_='js-tweet-text-container')
    
    url4 = 'https://space-facts.com/mars/'
    tables = pd.read_html(url4)
    df = tables[0]
    df.columns = ['Mars-Earth_Comparison', 'Mars', 'Earth']
    html_table1 = df.to_html()
    html_table2 = html_table1.replace('\n', '')
    mars_collection["fact_table"] = html_table2
    
    hemisphere_img_urls = []
    
    url4 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
    response = requests.get(url4)
    soup3 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['cerebrus_img'] = soup3.find('img', class_='wide-image')['src']
    mars_collection['cerebrus_title'] = soup3.find('h2', class_='title')

    
    url5 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'
    response = requests.get(url5)
    soup4 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['schiaparelli_img'] = soup4.find('img', class_='wide-image')['src']
    mars_collection['schiaparelli_title'] = soup4.find('h2', class_='title')
    
    url6 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'
    response = requests.get(url6)    
    soup5 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['syrtis_major_img'] = soup5.find('img', class_='wide-image')['src']
    mars_collection['syrtis_major_title'] = soup5.find('h2', class_='title')

    
    url7 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'
    response = requests.get(url7)
    soup6 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['valles_marineris_img'] = soup6.find('img', class_='wide-image')['src']
    mars_collection['valles_marineris_title'] = soup6.find('h2', class_='title')


    return mars_collection    

In [72]:
scrape()

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover',
 'news_p': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.",
 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA09113_ip.jpg',
 'mars_weather': <div class="js-tweet-text-container">
 <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="und">Weeeeee!  <a class="twitter-timeline-link" data-expanded-url="https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA23461" dir="ltr" href="https://t.co/terPoTqKM6" rel="nofollow noopener" target="_blank" title="https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA23461"><span class="tco-ellipsis"></span><span class="invisible">https://www.</span><span class="js-display-url">jpl.nasa.gov/spaceimages/de</span><span class="invisible">tails.php?id=PIA23461</span><span class="tco-ellipsis"><span class="invisible"> </span>…</span></a><a class="t

In [83]:
import pymongo

In [84]:
mars_scrape = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = mars_scrape["mars_db"]

In [87]:
mycol = mydb["mars_data"]
yy=scrape()

In [89]:
print(yy)

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover', 'news_p': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.", 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16729_ip.jpg', 'mars_weather': <div class="js-tweet-text-container">
<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="und">Weeeeee!  <a class="twitter-timeline-link" data-expanded-url="https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA23461" dir="ltr" href="https://t.co/terPoTqKM6" rel="nofollow noopener" target="_blank" title="https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA23461"><span class="tco-ellipsis"></span><span class="invisible">https://www.</span><span class="js-display-url">jpl.nasa.gov/spaceimages/de</span><span class="invisible">tails.php?id=PIA23461</span><span class="tco-ellipsis"><span class="invisible"> </span>…</span></a><a class="twitt

In [90]:
x = mycol.insert_one(yy)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
print(x)